# Execute VIP Jobs from your Computer

This Notebook uses the `VipSession` class from the [VIP Python client](https://github.com/virtual-imaging-platform/VIP-python-client/), which allows to run VIP applications on local datasets. In this tutorial you will learn how to:
- **Create** a VipSession instance;
- **Perform** an *upload-run-download* procedure (see image below) to get your outputs from VIP;
- **Parallelize** your executions on VIP;
- **Remove** your temporary data from VIP;
- **Resume & repeat** previous VIP executions.

For more advanced use of the Python client (*e.g.*, running different pipelines on the same dataset), please [read the doc on GitHub](https://github.com/virtual-imaging-platform/VIP-python-client/#vipsession).

<img src="../../imgs/Upload_Run_Download.png" alt="Procedure" height="250" title="Procedure for the Python Client"/>


*__N.B.__: In this Notebook, commands starting with "`!`" will only run on Linux distributions (including the Binder instance).*

In [ ]:
# Python libraries used in this tutorial
from pathlib import *
# VipSession class
import sys; sys.path.append('../..')
from src.VipSession import VipSession

## Initiate your Session 
*The VIP API key can be genereated in your own account settings. This short procedure can be found in the [documentation](https://github.com/virtual-imaging-platform/VIP-python-client/#get-a-vip-api-key).* 

In [ ]:
# Handshake with VIP
VipSession.init(api_key="VIP_API_KEY"); # Paste your VIP API key here

*VIP jobs will be launched through a persistent `VipSession` instance. Providing a session name is a good practice.*

In [ ]:
# Instantiate a VipSession object with session name: "Demo-VipSession"
my_session = VipSession("Demo-VipSession")

## The *upload-run-download* procedure

### Upload your dataset on VIP

*Local folder `./data` contains the entire dataset*

In [ ]:
input_dir = Path("data")
! tree {input_dir}

The VipSession method `upload_inputs()` is meant to upload the full dataset on VIP

In [ ]:
my_session.upload_inputs(input_dir);

### Launch the Application

#### Get the Pipeline Identifier

Search for available applications on VIP 

In [ ]:
VipSession.show_pipeline("cquest")

*If the previous cell displays "`No pipeline found`", please subscribe to the MR Spectroscopy group in your __account settings__ on the [VIP portal](https://vip.creatis.insa-lyon.fr/).*

In [ ]:
# Pipeline identifier = App/Version (/!\ mind the case)
pipeline_id = "CQUEST/0.2-egi"

#### Get your Input Settings

Show the pipeline description to know which inputs are required by the application

In [ ]:
VipSession.show_pipeline(pipeline_id)

Provide the input settings as a dictionnary

In [ ]:
# Input values can take several formats:
input_settings = { 
    "zipped_folder" : "data/basis.zip", # String
    "parameter_file" : input_dir / "parameters.txt", # PathLib object
    "data_file" : [file for file in (input_dir/"signals").iterdir()] # List
}

*A **list** of values (`[...]`) submits __parallel__ jobs on VIP. In this example, all signals (`data_file`) will be processed in parallel with the same `parameter_file` and `zipped_folder`.*

In [ ]:
# Display the previous settings as strings to see the list of files in `data_file`
import json
print("input_settings =",
    json.dumps(indent=3, obj={
        key: [str(v) for v in value] if isinstance(value, list) else str(value) for key, value in input_settings.items()
        }
    )
)

### Launch & monitor executions on VIP

VIP executions are launched with `launch_pipeline()`. Parallel *jobs* (*e.g.* processing the signals) are launched at once and grouped in a single *workflow*.

In [ ]:
my_session.launch_pipeline(pipeline_id, input_settings);

*You can monitor the workflow progression on https://vip.creatis.insa-lyon.fr/* ...

... or wait until all jobs are over on this terminal with `monitor_workflows()`:

In [ ]:
my_session.monitor_workflows(refresh_time=10);

### Download your Results

When all jobs are over, the outputs are downloaded at once with `download_outputs()`. Output files that have already been downloaded will be ignored.

In [ ]:
my_session.download_outputs();

*If `output_dir` was not specified when instanciating `my_session`, the outputs are stored at default location (recommended for beginners)*

In [ ]:
! tree {my_session.output_dir}

### Comments
1. This 4-step procedure (`upload_inputs()` -> `launch_pipeline()` -> `monitor_workflows()` -> `download_outputs()`) can be performed with a single command using `run_session()`:
```python 
    my_session = VipSession(
        session_name = "Demo-VipSession",
        input_dir = input_dir,
        pipeline_id = pipeline_id,
        input_settings = input_settings
    ).run_session(refresh_time=10);
```
2. Setting all properties (*e.g.*, `input_dir`, `pipeline_id`, `input_settings`) at instantiation (like above) allows early detection of common mistakes (*e.g.*, missing parameters or input files) before running VIP executions.

## Remove temporary data from VIP

*After the download, **your input and output data are still on VIP** (https://vip.creatis.insa-lyon.fr/)*

In [ ]:
print("Inputs are in:", my_session.vip_input_dir, "\n", 
      "Outputs are in:", my_session.vip_output_dir)

Please remove the temporary data from VIP with `finish()`

In [ ]:
my_session.finish();

## Use the session backups

### Check the backup file

*After each step, session data are automatically backed up in its output directory* 

In [ ]:
! tree {Path(my_session.output_dir).parent}

*The JSON file `session_data.json` contains everything you need to repeat the same executions on VIP*

In [ ]:
! head {Path(my_session.output_dir) / "session_data.json"} 

### Repeat previous executions

You can restore a previous VipSession instance using its session name

In [ ]:
new_session = VipSession("Demo-VipSession") # Name of the previous session

Use the `run_session()` shortcut to launch the full *Upload-Run-Download* procedure from this new VipSession instance

In [ ]:
new_session.run_session(refresh_time=10);

Check the output files

In [ ]:
! tree {new_session.output_dir}

## End this tutorial

The output data downloaded on your computer is yours to remove

In [14]:
! rm -r vip_outputs